In [ ]:
import numpy as np
import pandas as pd
import seaborn; seaborn.set()
from pmdarima import auto_arima
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import matplotlib.pylab as plt

In [ ]:
btcDec_df = pd.read_csv('../../Resources/BTC/BTC-USD_Historical_Dec-11-2021.csv',index_col='Date', parse_dates= True)
btcDec_df.head(5)

In [ ]:
btcDec_df.dtypes

In [ ]:
btcDec_df = btcDec_df.drop(columns=['Open','High','Low','Adj Close','Volume'])

In [ ]:
btcDec_df.head(10)

In [ ]:
btcDec_df.dtypes

In [ ]:
results = seasonal_decompose(btcDec_df['Close'],model ='multiplicative',period=1)
plt.rcParams.update({'figure.figsize': (15,10)})
results.plot()

In [ ]:
#Arima
import warnings
warnings.filterwarnings("ignore")

# Fit auto_arima function to AirPassengers dataset
stepwise_fit = auto_arima(btcDec_df['Close'], start_p = 0, start_q = 0,
max_p = 4, max_q = 4, m = 12,
start_P = 0, seasonal = True,
d = None, D = 1, trace = True,
error_action ='ignore', # we don't want to know if an order does not work
suppress_warnings = True, # we don't want convergence warnings
stepwise = True)# set to stepwise

# To print the summary
stepwise_fit.summary()


In [ ]:
# Split data into train / test sets
train = btcDec_df.iloc[:len(btcDec_df)-12]
test = btcDec_df.iloc[len(btcDec_df)-12:] # set one year(12 months) for testing

# Fit a SARIMAX(0, 1, 1)x(2, 1, 1, 12) on the training set
from statsmodels.tsa.statespace.sarimax import SARIMAX

model = SARIMAX(train['Close'],
order = (0, 1, 1),
seasonal_order =(2, 1, 1, 12))

result = model.fit()
result.summary()


In [ ]:
#Predictions of ARIMA Model agains the Test set
start = len(train)
end = len(train) + len(test) - 1

# Predictions for one-year against the test set
predictions = result.predict(start, end,typ = 'levels').rename("Close")

# plot predictions and actual values
predictions.plot(legend = True)
test['Close'].plot(legend = True)


In [ ]:
#Evaluate the model using MSE and RMSE
# Load specific evaluation tools
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse

# Calculate root mean squared error
rmse(test["Close"], predictions)

In [ ]:
# Calculate mean squared error
mean_squared_error(test["Close"], predictions)

In [ ]:
# Train the model on the full dataset
model = model = SARIMAX(btcDec_df['Close'],order = (0, 1, 1),seasonal_order =(2, 1, 1, 12))
result = model.fit()

# Forecast for the next 3 years
forecast = result.predict(start = len(btcDec_df),end = (len(btcDec_df)-1) + 3 * 12,typ = 'levels').rename('Forecast')

# Plot the forecast values
btcDec_df['Close'].plot(figsize = (20, 10), legend = True, fontsize =15)
forecast.plot(legend = True)
plt.title("Bitcoin Future Forcast after December First week", fontsize = 25)
plt.xlabel('Years',fontsize=15)
plt.ylabel('Closing Amount (USD)',fontsize=15)
